In [52]:
%load_ext autoreload
%autoreload 2

import keras
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from Jlayers import SimpleLayer
from utilsSimpleConv2D import*
from SpectralLayer import Spectral
from spectralconvolutions import *
from tensorflow.keras.layers import Dense,Layer
from typing import Tuple,List,Any,Dict
from tensorflow.python.keras import activations, initializers, regularizers, constraints

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
## Data

In [19]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train, x_test=x_train.reshape(-1,28,28,1), x_test.reshape(-1,28,28,1)

x_train, x_test = x_train / 255.0, x_test / 255.0

flat_train = np.reshape(x_train, [x_train.shape[0], 28 * 28])
flat_test = np.reshape(x_test, [x_test.shape[0], 28 * 28])

In [20]:
## parameters

In [21]:
spectral_config={
    'is_base_trainable' : False,
    'is_diag_start_trainable' : False,
    'is_diag_end_trainable' : True,
}

In [22]:
parameters_one={ "use_lambda_out":False,
                 "use_lambda_in":False,
                 "trainable_SM_kernel":True,
                 "use_bias":False,
                 "kernel_initializer":"glorot_uniform"}

In [23]:
## Linear Conv2D: Compact Matrix Ac

In [54]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Input(shape=(28,28,1)))


model1.add(SpectralConv2D_one(filters=1, kernel_size=5, strides=1, padding="VALID", activation=None,**parameters_one))
model1.add(keras.layers.MaxPooling2D((2,2)))
      

model1.add(SpectralConv2D_one(filters=1, kernel_size=3, strides=1, padding="VALID", activation=None,**parameters_one))
model1.add(keras.layers.MaxPooling2D((2,2)))

                   
model1.add(keras.layers.Flatten())     

model1.add(Spectral(1000, **spectral_config,use_bias=True, activation='relu'))
model1.add(Spectral(10, **spectral_config,use_bias=False, activation='softmax'))
           
opt = tf.keras.optimizers.Adam(learning_rate=0.03)
model1.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spectral_conv2d_one_38 (Sp  (None, 24, 24, 1)         25        
 ectralConv2D_one)                                               
                                                                 
 max_pooling2d_38 (MaxPooli  (None, 12, 12, 1)         0         
 ng2D)                                                           
                                                                 
 spectral_conv2d_one_39 (Sp  (None, 10, 10, 1)         9         
 ectralConv2D_one)                                               
                                                                 
 max_pooling2d_39 (MaxPooli  (None, 5, 5, 1)           0         
 ng2D)                                                           
                                                                 
 flatten_19 (Flatten)        (None, 25)              

In [55]:
accuracy=list()
epochs = 10
batch_size=512

history1 = model1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1)
accuracy.append(model1.evaluate(x_test,y_test,batch_size=batch_size,verbose="auto"))

Epoch 1/10
118/118 [==============================] - 53s 440ms/step - loss: 0.8271 - accuracy: 0.7388
Epoch 2/10
118/118 [==============================] - 61s 514ms/step - loss: 0.3523 - accuracy: 0.8931
Epoch 3/10
118/118 [==============================] - 84s 715ms/step - loss: 0.3256 - accuracy: 0.9017
Epoch 4/10
118/118 [==============================] - 87s 740ms/step - loss: 0.3040 - accuracy: 0.9075
Epoch 5/10
118/118 [==============================] - 84s 712ms/step - loss: 0.2933 - accuracy: 0.9108
Epoch 6/10
118/118 [==============================] - 87s 737ms/step - loss: 0.2825 - accuracy: 0.9147
Epoch 7/10
118/118 [==============================] - 88s 743ms/step - loss: 0.2775 - accuracy: 0.9153
Epoch 8/10
118/118 [==============================] - 92s 777ms/step - loss: 0.2702 - accuracy: 0.9183
Epoch 9/10
118/118 [==============================] - 89s 753ms/step - loss: 0.2710 - accuracy: 0.9176
Epoch 10/10
20/20 [==============================] - 3s 71ms/step - loss: